In [4]:
import sqlite3
import xml.etree.ElementTree as ET
import os
import re

# Connect to the SQLite database
conn = sqlite3.connect("Learning_Database.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute('DROP TABLE IF EXISTS variables')

# Adjust the database schema to include variable_name and variable_value
cursor.execute('''
    CREATE TABLE variables (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        session_data_id TEXT NOT NULL,
        file_name TEXT NOT NULL,
        Pitch TEXT,
        variable_name TEXT NOT NULL,
        variable_value REAL,
        variable_component TEXT
    )
''')

# Commit the changes before proceeding to parsing XML files
conn.commit()

# Define the directory path and component lists
directory_path = "G:\\My Drive\\Pitching Data\\Kinematic Data\\Pro"
X_list = ['Lead_Knee_Angle@Footstrike', 'Lead_Knee_Angle@Release', 'Lead_Leg_GRF_max', 'Lead_Leg_GRF_min', 'Pitching_Shoulder_Angle@Footstrike', '{elvis_Ang_Vel_max', 'Thorax_Ang_Vel_max', 'Pitching_Humerus_Ang_Vel_Max']  
Y_list = ['MaxPelvisLinearVel_MPH', 'Lead_Knee_Angle@Footstrike','Lead_Knee_Angle@Release', 'Pelvis_Angle@Footstrike', 'Pelvis_Angle@Release', 'Lead_Leg_GRF_min', 'Trunk_wrt_Pelvis_FE@Release'] 
Z_list = ['Hip Shoulders Sep@Footstrike', 'Pelvis_Angle@Footstrike', 'Trunk_Angle@Footstrike', 'Lead_Leg_GRF_max', 'Pitching_Shoulder_Angle@Footstrike', 'Pitching_Shoulder_Angle_Max']  

while True:
    xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]
    new_data_entries = 0

    for file_name in xml_files:
        try:
            cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
            count = cursor.fetchone()[0]

            if count == 0:
                tree = ET.parse(file_name)
                root = tree.getroot()

                session_data_id_match = re.search(r"session_data_(.+)\.xml", file_name)
                session_data_id = session_data_id_match.group(1) if session_data_id_match else None

                if session_data_id is not None:
                    pitch_value = None
                    for owner_element in root.findall('.//owner'):
                        if 'Fastball' in owner_element.get('value', ''):
                            pitch_value = owner_element.get('value', '')
                            break  # Assuming you only need the first 'Fastball' occurrence

                    for variable_element in root.findall('.//name[@value]'):
                        variable_name = variable_element.attrib['value']
                        for component in ['X', 'Y', 'Z']:
                            component_element = variable_element.find(f'./component[@value="{component}"]')
                            if component_element is not None and variable_name in eval(f"{component}_list"):
                                try:
                                    variable_value = float(component_element.attrib['data'].replace(',', '.'))
                                    # Insert each variable into the database
                                    cursor.execute('''
                                        INSERT INTO variables (
                                            session_data_id, file_name, Pitch, variable_name, variable_value, variable_component
                                        ) VALUES (?, ?, ?, ?, ?, ?)
                                    ''', (session_data_id, file_name, pitch_value, variable_name, variable_value, component))
                                except ValueError:
                                    continue  # Skip if conversion to float fails

                    new_data_entries += 1

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

    conn.commit()

    if new_data_entries == 0:
        break

conn.close()
print("Data inserted into the database.")


Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_SB_06-23-22.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_CH_10-12-23.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_WM_11-21-23.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_TF_06-02-23.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_JL_02-15-23.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_MS_08-24-23.xml: sequence item 0: expected str instance, int found
Error processing file G:\My Drive\Pitching Data\Kinematic Data\Pro\session_data_RT_06-05-23.xml: sequence item 0: expe

In [4]:
import sqlite3
import xml.etree.ElementTree as ET
import os
import re

# Connect to the SQLite database
conn = sqlite3.connect("Learning_Database.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute('DROP TABLE IF EXISTS variables')

# Adjust the database schema to include each variable as its own column
cursor.execute('''
    CREATE TABLE variables (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        session_data_id TEXT NOT NULL,
        file_name TEXT NOT NULL,
        Pitch TEXT,
        Lead_Knee_Angle_Footstrike_X REAL,
        Lead_Knee_Angle_Release_X REAL,
        Lead_Leg_GRF_max_X REAL,
        Lead_Leg_GRF_min_X REAL,
        Pitching_Shoulder_Angle_Footstrike_X REAL,
        Elvis_Ang_Vel_max_X REAL,
        Thorax_Ang_Vel_max_X REAL,
        Pitching_Humerus_Ang_Vel_Max_X REAL,
        MaxPelvisLinearVel_MPH_Y REAL,
        Lead_Knee_Angle_Footstrike_Y REAL,
        Lead_Knee_Angle_Release_Y REAL,
        Pelvis_Angle_Footstrike_Y REAL,
        Pelvis_Angle_Release_Y REAL,
        Lead_Leg_GRF_min_Y REAL,
        Trunk_wrt_Pelvis_FE_Release_Y REAL,
        Hip_Shoulders_Sep_Footstrike_Z REAL,
        Pelvis_Angle_Footstrike_Z REAL,
        Trunk_Angle_Footstrike_Z REAL,
        Lead_Leg_GRF_max_Z REAL,
        Pitching_Shoulder_Angle_Footstrike_Z REAL,
        Pitching_Shoulder_Angle_Max_Z REAL
    )
''')

# Commit the changes before proceeding to parsing XML files
conn.commit()

# Define the directory path
directory_path = "G:\\My Drive\\Pitching Data\\Kinematic Data\\Pro"

while True:
    xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]
    new_data_entries = 0

    for file_name in xml_files:
        try:
            cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
            count = cursor.fetchone()[0]

            if count == 0:
                tree = ET.parse(file_name)
                root = tree.getroot()

                session_data_id_match = re.search(r"session_data_(.+)\.xml", file_name)
                session_data_id = session_data_id_match.group(1) if session_data_id_match else None

                if session_data_id is not None:
                    pitch_value = None
                    for owner_element in root.findall('.//owner'):
                        if 'Fastball' in owner_element.get('value', ''):
                            pitch_value = owner_element.get('value', '')
                            break  # Assuming you only need the first 'Fastball' occurrence

                    # Initialize a dictionary with None values for each variable
                    variables_data = {column[0]: None for column in cursor.execute('PRAGMA table_info(variables)').fetchall() if column[1] not in ['id', 'session_data_id', 'file_name', 'Pitch']}
                    
                    for variable_element in root.findall('.//name[@value]'):
                        variable_name = variable_element.attrib['value']
                        for component in ['X', 'Y', 'Z']:
                            component_element = variable_element.find(f'./component[@value="{component}"]')
                            if component_element is not None:
                                full_column_name = f"{variable_name}_{component}"
                                if full_column_name in variables_data.keys():
                                    try:
                                        variables_data[full_column_name] = float(component_element.attrib['data'].replace(',', '.'))
                                    except ValueError:
                                        continue  # Skip if conversion to float fails

                    # Assuming variables_data is correctly populated

                    # Ensure column names are correctly formatted - this step might be unnecessary if all column names are simple strings without spaces or special characters
                    insert_columns = ', '.join([f'"{column}"' for column in variables_data.keys()])  # Enclose column names in quotes
                    
                    # Prepare the values for insertion, including session_data_id, file_name, pitch_value, and all variable values
                    insert_values = [session_data_id, file_name, pitch_value] + list(variables_data.values())
                    
                    # Immediately before constructing the sql_command, define placeholders for dynamic SQL execution
                    placeholders = ', '.join(['?'] * (len(variables_data) + 3))  # +3 for session_data_id, file_name, Pitch
                    
                    sql_command = f'''
                        INSERT INTO variables (
                            session_data_id, file_name, Pitch, {insert_columns}
                        ) VALUES (?, ?, ?, {placeholders})
                    '''

                    try:
                        cursor.execute(sql_command, insert_values)
                    except Exception as e:
                        print(f"Error executing SQL: {e}\nSQL Command: {sql_command}\nValues: {insert_values}")
        finally:
            print("Attempted database operation")
                   
    conn.commit()

    if new_data_entries == 0:
        break

conn.close()
print("Data inserted into the database.")


Error executing SQL: table variables has no column named 4
SQL Command: 
                        INSERT INTO variables (
                            session_data_id, file_name, Pitch, "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    
Values: ['SB_06-23-22', 'G:\\My Drive\\Pitching Data\\Kinematic Data\\Pro\\session_data_SB_06-23-22.xml', 'Fastball RH 10.c3d', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Attempted database operation
Error executing SQL: table variables has no column named 4
SQL Command: 
                        INSERT INTO variables (
                            session_data_id, file_name, Pitch, "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"